In [1]:
import numpy as np
import pandas as pd
import nltk
from nltk import word_tokenize, FreqDist
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt
nltk.download
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')
from nltk.tokenize import TweetTokenizer
from nltk.stem.porter import *
from keras.preprocessing import sequence
from keras.models import load_model
import preprocessor as p

import re

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Stack\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Stack\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Stack\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [5]:
df_train = pd.read_csv('clean_train_data_big.csv')

In [3]:
df_train = pd.read_csv('train_tweets.csv')
df_train.drop('id', axis=1, inplace=True)

In [6]:
df_train.head()

,target,tweet
0,1.0,@TBthatsme what!!!! Why didn't you tell us? Uh...
1,1.0,@stuartsharpe Ah you're saying being obsessed ...
2,0.0,@lejjewellery is it your birthday today? :o ye...
3,1.0,"@TerrenceJ106 Hey babe follow me,love u on da ..."
4,0.0,"working hard as always on TNARevolution.com, t..."


In [7]:
def preprocess_data(data):
  #Removes Numbers
    stemmer = PorterStemmer()
    data = data.astype(str).str.replace('\d+', '')
    lower_text = data.str.lower()
    lemmatizer = nltk.stem.WordNetLemmatizer()
    w_tokenizer =  TweetTokenizer()
    
    def stemming(text):
        return [stemmer.stem(w) for w in text]
    
    def lemmatize_text(text):
        return [(lemmatizer.lemmatize(w)) for w in w_tokenizer.tokenize((text))]
    
    def remove_punctuation(words):
        new_words = []
        for word in words:
            new_word = re.sub(r'[^\w\s]', '', (word))
            if new_word != '':
                new_words.append(new_word)
        return new_words
    
    words = lower_text.apply(lemmatize_text)
    words = words.apply(remove_punctuation)
    words = words.apply(stemming)
    
    
    return pd.DataFrame(words)

def preprocess(df):
    
    df['hashtag'] = df['tweet'].apply(lambda x: re.findall(r"#(\w+)", x))
    df['text'] = 0
    for i in range(0,len(df)):
        df['text'][i] = p.clean(df['tweet'][i])
        
    df['text_helper'] = df['text'].copy()
    df['text_helper'] = preprocess_data(df['text_helper']) 
    
    stop_words = set(stopwords.words('english'))
    df['text_helper'] = df['text_helper'].apply(lambda x: [item for item in x if item not in stop_words])
    
    a = len(max(np.array(df_train['text_helper']), key=len))
    
    
    def convert_to_string(listWord):
        return ' '.join(listWord)
    df['text_helper'] = df['text_helper'].apply(convert_to_string)
   
    return a
    


In [9]:
max_tweet_word_count = preprocess(df_train)

print(max_tweet_word_count)

C:\Users\Stack\AppData\Local\Temp\ipykernel_856\3072535002.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'][i] = p.clean(df['tweet'][i])
C:\Users\Stack\AppData\Local\Temp\ipykernel_856\3072535002.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  data = data.astype(str).str.replace('\d+', '')


36


In [11]:
df_train.to_csv("preprocessed_tweets.csv", index=False)

In [3]:
df_train = pd.read_csv('preprocessed_tweets.csv')
max_tweet_word_count = 36

In [5]:
df_train.head()

,target,tweet,hashtag,text,text_helper
0,1.0,@TBthatsme what!!!! Why didn't you tell us? Uh...,[],what!!!! Why didn't you tell us? Uhhh... You s...,whi didnt tell u uhhh suck haha im kid got hom...
1,1.0,@stuartsharpe Ah you're saying being obsessed ...,[],Ah you're saying being obsessed with his body ...,ah say obsess hi bodi languag onli thing keep ...
2,0.0,@lejjewellery is it your birthday today? :o ye...,[],"is it your birthday today? yeah, im worries no...",birthday today yeah im worri im go get humunga...
3,1.0,"@TerrenceJ106 Hey babe follow me,love u on da ...",[],"Hey babe follow me,love u on da show",hey babe follow love u da show
4,0.0,"working hard as always on TNARevolution.com, t...",[],"working hard as always on TNARevolution.com, t...",work hard alway tnarevolutioncom think time sl...


In [11]:
list_of_words = []
empty_indexes = []
for i in range(0,len(df_train['text_helper'])):
    try:
        list_of_words.extend(df_train['text_helper'][i].split())
    except:
        empty_indexes.append(i)
    
total_words = len(list(set(list_of_words)))

In [13]:
df_train = df_train.drop(empty_indexes)
df_train = df_train.reset_index()

In [17]:
df_train

,index,target,tweet,hashtag,text,text_helper
0,0,1.0,@TBthatsme what!!!! Why didn't you tell us? Uh...,[],what!!!! Why didn't you tell us? Uhhh... You s...,whi didnt tell u uhhh suck haha im kid got hom...
1,1,1.0,@stuartsharpe Ah you're saying being obsessed ...,[],Ah you're saying being obsessed with his body ...,ah say obsess hi bodi languag onli thing keep ...
2,2,0.0,@lejjewellery is it your birthday today? :o ye...,[],"is it your birthday today? yeah, im worries no...",birthday today yeah im worri im go get humunga...
3,3,1.0,"@TerrenceJ106 Hey babe follow me,love u on da ...",[],"Hey babe follow me,love u on da show",hey babe follow love u da show
4,4,0.0,"working hard as always on TNARevolution.com, t...",[],"working hard as always on TNARevolution.com, t...",work hard alway tnarevolutioncom think time sl...
...,...,...,...,...,...,...
1592339,1599994,0.0,@ScruffyPanther when's the last exam mdear? X,[],when's the last exam mdear? X,last exam mdear x
1592340,1599995,0.0,Aaa lunch time. So loaded w work but its ok. ...,[],Aaa lunch time. So loaded w work but its ok. M...,aaa lunch time load w work ok mani peopl arent...
1592341,1599996,1.0,I'm watching the movie ''The curious case of B...,[],I'm watching the movie ''The curious case of B...,im watch movi curiou case benjamin button good...
1592342,1599997,1.0,the scene where amanda peets walks into the gl...,[],the scene where amanda peets walks into the gl...,scene amanda peet walk glass door lot like lov...


Code fo the NN starts here

In [10]:
df_train.to_csv("preprocessed_big_cleaned_tweets_2.csv", index=False)

## Loading Data

In [4]:
df_train = pd.read_csv("preprocessed_big_cleaned_tweets_2.csv")

In [1]:
import gensim.downloader
glove_vectors = gensim.downloader.load('glove-twitter-25')

[==================================================] 100.0% 66.0/66.0MB downloaded


In [5]:
vocabulary_size, embedding_size = glove_vectors.vectors.shape
MAX_WORDS = 50
print("Vocabulary Size: {} - Embedding Dim: {} - Max Words: {}".format(vocabulary_size, embedding_size, MAX_WORDS))

Vocabulary Size: 1193514 - Embedding Dim: 25 - Max Words: 50


In [6]:
def text_splitter_tokenizer(text):
    words = text.split()
    tokenized_word = []
    for word in words:
        try:
            tokenized_word.append(glove_vectors.key_to_index[word])
        except KeyError:
            tokenized_word.append(0)
    return np.array(tokenized_word)        

In [7]:
X_tokenized = np.array(df_train["text_helper"].apply(text_splitter_tokenizer))
X_padded = pad_sequences(X_tokenized, maxlen = MAX_WORDS, padding = 'post', truncating = 'post')

In [8]:
y = np.array(df_train['target'])

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X_padded, y, test_size=0.7, random_state=42)

In [106]:
from keras.models import Sequential
from keras.layers import Embedding,Bidirectional, LSTM, Dense, Dropout
from keras.optimizers import adam_v2

learning_rate = 0.01

model1 = Sequential([
    Embedding(input_dim=vocabulary_size,output_dim=embedding_size,weights=[glove_vectors.vectors],input_length=MAX_WORDS,mask_zero=True,trainable=False),
    Dropout(0.5),
    Bidirectional(LSTM(64,dropout=0.1 , return_sequences=True)),
    Bidirectional(LSTM(40,dropout=0.4)),
    Dense(64, activation='tanh'),
    Dense(16, activation='tanh'),
    Dropout(0.5),
    Dense(4, activation='tanh'),
    Dense(1, activation='sigmoid')
])

model1.summary()
model1.compile(optimizer=adam_v2.Adam(learning_rate), loss='binary_crossentropy', metrics=['acc'])

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 50, 25)            29837850  
_________________________________________________________________
dropout_6 (Dropout)          (None, 50, 25)            0         
_________________________________________________________________
bidirectional_6 (Bidirection (None, 50, 128)           46080     
_________________________________________________________________
bidirectional_7 (Bidirection (None, 80)                54080     
_________________________________________________________________
dense_12 (Dense)             (None, 64)                5184      
_________________________________________________________________
dense_13 (Dense)             (None, 16)                1040      
_________________________________________________________________
dropout_7 (Dropout)          (None, 16)               

In [109]:
model2 = Sequential([
    Embedding(input_dim=vocabulary_size,output_dim=embedding_size,weights=[glove_vectors.vectors],input_length=MAX_WORDS,mask_zero=True,trainable=False),
    Dropout(0.5),
    LSTM(512, dropout=0.2),
    Dense(1, activation='sigmoid')
])

model2.summary()
model2.compile(optimizer=adam_v2.Adam(learning_rate), loss='binary_crossentropy', metrics=['acc'])

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 50, 25)            29837850  
_________________________________________________________________
dropout_9 (Dropout)          (None, 50, 25)            0         
_________________________________________________________________
lstm_10 (LSTM)               (None, 512)               1101824   
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 513       
Total params: 30,940,187
Trainable params: 1,102,337
Non-trainable params: 29,837,850
_________________________________________________________________


In [113]:
learning_rate = 0.01

model3 = Sequential([
    Embedding(input_dim=vocabulary_size,output_dim=embedding_size,weights=[glove_vectors.vectors],input_length=MAX_WORDS,mask_zero=True,trainable=False),
    Dropout(0.5),
    Bidirectional(LSTM(64,dropout=0.1 , return_sequences=True)),
    Bidirectional(LSTM(40,dropout=0.4)),
    Dense(1, activation='sigmoid')
])

model3.summary()
model3.compile(optimizer=adam_v2.Adam(learning_rate), loss='binary_crossentropy', metrics=['acc'])

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 50, 25)            29837850  
_________________________________________________________________
dropout_10 (Dropout)         (None, 50, 25)            0         
_________________________________________________________________
bidirectional_10 (Bidirectio (None, 50, 128)           46080     
_________________________________________________________________
bidirectional_11 (Bidirectio (None, 80)                54080     
_________________________________________________________________
dense_18 (Dense)             (None, 1)                 81        
Total params: 29,938,091
Trainable params: 100,241
Non-trainable params: 29,837,850
_________________________________________________________________


In [117]:
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
model4=Sequential()
model4.add(Embedding(input_dim=vocabulary_size,output_dim=embedding_size,weights=[glove_vectors.vectors],input_length=MAX_WORDS,mask_zero=True,trainable=False))
#output(batch_size, 100, 300)
model4.add(Conv1D(128, 5, activation='relu'))
#output(batch_size, 96, 128), (batch_size, (size of input - size of kernel + 1) , number of filters)
model4.add(MaxPooling1D(pool_size=1,strides=2,padding='valid'))
#output(batch_size, 48, 128), (batch_size, (size of input - pool_size + 1)/strides, number of filters )
model4.add(LSTM(128,dropout=0.2)) #Adding 128 lstm neurons in the layer
#output(batch_size, 128)
model4.add(Dense(1,activation='sigmoid'))
model4.summary()
model4.compile(optimizer=adam_v2.Adam(learning_rate), loss='binary_crossentropy', metrics=['acc'])

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 50, 25)            29837850  
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 46, 128)           16128     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 23, 128)           0         
_________________________________________________________________
lstm_14 (LSTM)               (None, 128)               131584    
_________________________________________________________________
dense_20 (Dense)             (None, 1)                 129       
Total params: 29,985,691
Trainable params: 147,841
Non-trainable params: 29,837,850
_________________________________________________________________


In [120]:
learning_rate = 0.01

model6 = Sequential([
    Embedding(input_dim=vocabulary_size,output_dim=embedding_size,weights=[glove_vectors.vectors],input_length=MAX_WORDS,mask_zero=True,trainable=False),
    Dropout(0.5),
    Conv1D(128, 5, activation='relu'),
    MaxPooling1D(pool_size=1,strides=2,padding='valid'),
    Bidirectional(LSTM(64,dropout=0.1 , return_sequences=True)),
    Bidirectional(LSTM(40,dropout=0.4)),
    Dense(1, activation='sigmoid')
])

model6.summary()
model6.compile(optimizer=adam_v2.Adam(learning_rate), loss='binary_crossentropy', metrics=['acc'])

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 50, 25)            29837850  
_________________________________________________________________
dropout_11 (Dropout)         (None, 50, 25)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 46, 128)           16128     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 23, 128)           0         
_________________________________________________________________
bidirectional_12 (Bidirectio (None, 23, 128)           98816     
_________________________________________________________________
bidirectional_13 (Bidirectio (None, 80)                54080     
_________________________________________________________________
dense_21 (Dense)             (None, 1)                

In [129]:
learning_rate = 0.001

model7 = Sequential([
    Embedding(input_dim=vocabulary_size,output_dim=embedding_size,weights=[glove_vectors.vectors],input_length=MAX_WORDS,mask_zero=True,trainable=False),
    Dropout(0.5),
    Conv1D(128, 5, activation='relu'),
    MaxPooling1D(pool_size=1,strides=2,padding='valid'),
    Bidirectional(LSTM(64,dropout=0.1 , return_sequences=True)),
    Bidirectional(LSTM(40,dropout=0.4)),
    Dropout(0.5),
    Dense(4, activation='sigmoid'),
    Dense(1, activation='sigmoid')
])

model7.summary()
model7.compile(optimizer=adam_v2.Adam(learning_rate), loss='binary_crossentropy', metrics=['acc'])

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, 50, 25)            29837850  
_________________________________________________________________
dropout_16 (Dropout)         (None, 50, 25)            0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 46, 128)           16128     
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 23, 128)           0         
_________________________________________________________________
bidirectional_18 (Bidirectio (None, 23, 128)           98816     
_________________________________________________________________
bidirectional_19 (Bidirectio (None, 80)                54080     
_________________________________________________________________
dropout_17 (Dropout)         (None, 80)              

In [110]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

# Specify training parameters: batch size and number of epochs
batch_size = 64
num_epochs = 5

# Stop training if the validation loss doesn't fall for 3 consecutive epochs
earlystopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
# Train the model, reserve some training data for validation
hist1 = model1.fit(X_train, y_train, batch_size=batch_size, epochs=num_epochs,
          validation_split=0.2, callbacks=[earlystopping], 
          verbose=1, shuffle=True)

Epoch 1/5
993/993 [==============================] - 177s 160ms/step - loss: 0.6784 - acc: 0.5757 - val_loss: 0.6659 - val_acc: 0.6229
Epoch 2/5
993/993 [==============================] - 151s 152ms/step - loss: 0.6709 - acc: 0.5965 - val_loss: 0.6568 - val_acc: 0.6175
Epoch 3/5
993/993 [==============================] - 157s 158ms/step - loss: 0.6658 - acc: 0.6012 - val_loss: 0.6562 - val_acc: 0.6264
Epoch 4/5
993/993 [==============================] - 158s 159ms/step - loss: 0.6625 - acc: 0.6081 - val_loss: 0.6605 - val_acc: 0.6161
Epoch 5/5
993/993 [==============================] - 152s 153ms/step - loss: 0.6623 - acc: 0.6067 - val_loss: 0.6652 - val_acc: 0.5916


In [111]:
hist2 = model2.fit(X_train, y_train, batch_size=batch_size, epochs=num_epochs,
          validation_split=0.2, callbacks=[earlystopping], 
          verbose=1, shuffle=True)

Epoch 1/5
993/993 [==============================] - 340s 338ms/step - loss: 0.6543 - acc: 0.6165 - val_loss: 0.6075 - val_acc: 0.6632
Epoch 2/5
993/993 [==============================] - 323s 325ms/step - loss: 0.6416 - acc: 0.6277 - val_loss: 0.6118 - val_acc: 0.6631
Epoch 3/5
993/993 [==============================] - 326s 328ms/step - loss: 0.6392 - acc: 0.6309 - val_loss: 0.6053 - val_acc: 0.6656
Epoch 4/5
993/993 [==============================] - 356s 358ms/step - loss: 0.6382 - acc: 0.6342 - val_loss: 0.6074 - val_acc: 0.6673
Epoch 5/5
993/993 [==============================] - 373s 376ms/step - loss: 0.6381 - acc: 0.6313 - val_loss: 0.6003 - val_acc: 0.6682


In [114]:
hist3 = model3.fit(X_train, y_train, batch_size=batch_size, epochs=num_epochs,
          validation_split=0.2, callbacks=[earlystopping], 
          verbose=1, shuffle=True)

Epoch 1/5
993/993 [==============================] - 213s 186ms/step - loss: 0.6388 - acc: 0.6325 - val_loss: 0.6009 - val_acc: 0.6699
Epoch 2/5
993/993 [==============================] - 180s 181ms/step - loss: 0.6290 - acc: 0.6444 - val_loss: 0.5988 - val_acc: 0.6809
Epoch 3/5
993/993 [==============================] - 179s 181ms/step - loss: 0.6237 - acc: 0.6478 - val_loss: 0.5788 - val_acc: 0.6960
Epoch 4/5
993/993 [==============================] - 177s 179ms/step - loss: 0.6197 - acc: 0.6536 - val_loss: 0.5757 - val_acc: 0.6915
Epoch 5/5
993/993 [==============================] - 177s 178ms/step - loss: 0.6174 - acc: 0.6544 - val_loss: 0.5741 - val_acc: 0.7003


In [118]:
hist4 = model4.fit(X_train, y_train, batch_size=batch_size, epochs=num_epochs,
          validation_split=0.2, callbacks=[earlystopping], 
          verbose=1, shuffle=True)

Epoch 1/5
993/993 [==============================] - 31s 28ms/step - loss: 0.6972 - acc: 0.4969 - val_loss: 0.6931 - val_acc: 0.4972
Epoch 2/5
993/993 [==============================] - 27s 28ms/step - loss: 0.6948 - acc: 0.4999 - val_loss: 0.6948 - val_acc: 0.5028
Epoch 3/5
993/993 [==============================] - 27s 28ms/step - loss: 0.6947 - acc: 0.4984 - val_loss: 0.6936 - val_acc: 0.4972
Epoch 4/5
993/993 [==============================] - 28s 28ms/step - loss: 0.6944 - acc: 0.4991 - val_loss: 0.6970 - val_acc: 0.5028


In [121]:
hist6 = model6.fit(X_train, y_train, batch_size=batch_size, epochs=num_epochs,
          validation_split=0.2, callbacks=[earlystopping], 
          verbose=1, shuffle=True)

Epoch 1/5
993/993 [==============================] - 70s 60ms/step - loss: 0.6655 - acc: 0.5996 - val_loss: 0.6429 - val_acc: 0.6440
Epoch 2/5
993/993 [==============================] - 58s 59ms/step - loss: 0.6543 - acc: 0.6150 - val_loss: 0.6360 - val_acc: 0.6450
Epoch 3/5
993/993 [==============================] - 58s 59ms/step - loss: 0.6533 - acc: 0.6163 - val_loss: 0.6360 - val_acc: 0.6468
Epoch 4/5
993/993 [==============================] - 58s 59ms/step - loss: 0.6530 - acc: 0.6161 - val_loss: 0.6335 - val_acc: 0.6464
Epoch 5/5
993/993 [==============================] - 59s 60ms/step - loss: 0.6510 - acc: 0.6189 - val_loss: 0.6300 - val_acc: 0.6516


In [130]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

# Specify training parameters: batch size and number of epochs
batch_size = 64
num_epochs = 10

# Stop training if the validation loss doesn't fall for 3 consecutive epochs
earlystopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
# Train the model, reserve some training data for validation
hist1 = model7.fit(X_train, y_train, batch_size=batch_size, epochs=num_epochs,
          validation_split=0.2, callbacks=[earlystopping], 
          verbose=1, shuffle=True)

Epoch 1/10
5954/5954 [==============================] - 456s 75ms/step - loss: 0.6527 - acc: 0.6119 - val_loss: 0.6050 - val_acc: 0.6739
Epoch 2/10
5954/5954 [==============================] - 424s 71ms/step - loss: 0.6314 - acc: 0.6393 - val_loss: 0.5880 - val_acc: 0.6899
Epoch 3/10
5954/5954 [==============================] - 442s 74ms/step - loss: 0.6245 - acc: 0.6468 - val_loss: 0.5845 - val_acc: 0.6943
Epoch 4/10
5954/5954 [==============================] - 432s 73ms/step - loss: 0.6197 - acc: 0.6520 - val_loss: 0.5781 - val_acc: 0.6955
Epoch 5/10
5954/5954 [==============================] - 327s 55ms/step - loss: 0.6156 - acc: 0.6557 - val_loss: 0.5664 - val_acc: 0.7029
Epoch 6/10
5954/5954 [==============================] - 323s 54ms/step - loss: 0.6121 - acc: 0.6597 - val_loss: 0.5641 - val_acc: 0.7052
Epoch 7/10
5954/5954 [==============================] - 322s 54ms/step - loss: 0.6095 - acc: 0.6621 - val_loss: 0.5601 - val_acc: 0.7119
Epoch 8/10
5954/5954 [===================

In [133]:
model7.save('./ultra_final_cnn_lstm.h5')

In [134]:
X_null, X_test, y_null, y_test = train_test_split(X_test, y_test, test_size=0.01, random_state=42)

In [135]:
scores = model7.evaluate(X_test, y_test, verbose=0)  # returns loss and other metrics specified in model.compile()
print("Test accuracy:", scores[1])  # scores[1] should correspond to accuracy that I passed in metrics=['accuracy']

Test accuracy: 0.7198128700256348


## Gradio Preprocess

In [3]:
new_model = load_model('./ultra_final_cnn_lstm.h5')

In [ ]:
scores = new_model.evaluate(X_test, y_test, verbose=0)  # returns loss and other metrics specified in model.compile()
print("Test accuracy:", scores[1])

In [66]:
def single_preprocess(text):
    
    text = p.clean(text)
    text = single_preprocess_data(text)
    stop_words = set(stopwords.words('english'))
    text = [word for word in text if not word in stop_words]
#     vectorizer = CountVectorizer(max_features=vocabulary_size, preprocessor=lambda x: x, tokenizer=lambda x: x)
    # Convert the features using .toarray() for a compact representation
    
    text = ' '.join(text)
    
    return text

def single_preprocess_data(data):
  #Removes Numbers
    stemmer = PorterStemmer()
    data = data.replace('\d+', '')
    lower_text = data.lower()
    lemmatizer = nltk.stem.WordNetLemmatizer()
    w_tokenizer =  TweetTokenizer()
    
    def stemming(text):
        return [stemmer.stem(w) for w in text]
    
    def lemmatize_text(text):
        return [(lemmatizer.lemmatize(w)) for w in w_tokenizer.tokenize((text))]
    
    def remove_punctuation(words):
        new_words = []
        for word in words:
            new_word = re.sub(r'[^\w\s]', '', (word))
            if new_word != '':
                new_words.append(new_word)
        return new_words
    
    words = lemmatize_text(lower_text)
    words = stemming(words)
    words = remove_punctuation(words)
    
    return words

In [86]:
def string_to_padded_tokens(raw_input_tweet,max_len):
    x_sequence = text_splitter_tokenizer(raw_input_tweet)
    zeros = np.zeros(max_len - len(x_sequence))
    return np.concatenate([x_sequence,zeros])
    

In [87]:
def text_splitter_tokenizer(text):
    words = text.split()
    tokenized_word = []
    for word in words:
        try:
            tokenized_word.append(glove_vectors.key_to_index[word])
        except KeyError:
            tokenized_word.append(0)
    return np.array(tokenized_word)        

In [138]:
raw_input_tweet = single_preprocess("jnetz praising giselle on this tiktok video and one comment says “as expected, our giselle")
testing_single_tweet =  string_to_padded_tokens(raw_input_tweet,50)
print(testing_single_tweet)

[0.00000e+00 6.01762e+05 1.07855e+05 1.42930e+04 3.76915e+05 2.86000e+02
 9.60000e+01 2.18600e+03 2.26000e+02 2.33700e+03 1.07855e+05 0.00000e+00
 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00
 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00
 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00
 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00
 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00
 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00
 0.00000e+00 0.00000e+00]


In [139]:
zeroarr = np.zeros(len(testing_single_tweet))

In [140]:
stacked_tweets = np.vstack((testing_single_tweet,zeroarr))

stacked_tweets

array([[0.00000e+00, 6.01762e+05, 1.07855e+05, 1.42930e+04, 3.76915e+05,
        2.86000e+02, 9.60000e+01, 2.18600e+03, 2.26000e+02, 2.33700e+03,
        1.07855e+05, 0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00],
       [0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 0.00000e+00, 0.00000e+00, 0.0

In [141]:
new_model.predict(stacked_tweets,batch_size=2,
    verbose=1)

1/1 [==============================] - 2s 2s/step


array([[0.57634217],
       [0.6497878 ]], dtype=float32)

## Gradio Implemetation

In [150]:
import gradio as gr

def question_answer(Tweet):
      
    raw_input_tweet = single_preprocess(Tweet)

    testing_single_tweet = string_to_padded_tokens(raw_input_tweet,50)
    
    zeroarr = np.zeros(len(testing_single_tweet))
    
    stacked_tweets = np.vstack((testing_single_tweet,zeroarr))

    results = new_model.predict(stacked_tweets,batch_size=2,
    verbose=1)
    
    confidence = results[0]
    
    if confidence < 0.4:
        answer = "Hate speech detected"
    else:
        if confidence < 0.6:
            answer = "Neutral speech detected"
        else:
            answer = "Positive speech"
            
    
    
    return (answer, confidence)
# Return a tuple consisting of two strings: (answer, confidence) 

iface = gr.Interface(fn=question_answer, 
                     inputs = gr.inputs.Textbox(lines=5, placeholder="Insert tweet here..."),
                     outputs=["textbox", "text"],theme="peach",
                     title="Tweet hate speech detector",
                     allow_flagging = "never",
                    ).launch()

Running on local URL:  http://127.0.0.1:7864/

To create a public link, set `share=True` in `launch()`.


1/1 [==============================] - 0s 26ms/step
